In [1]:
from dmrApp import create_app, db
from dmrApp.models import Dmrs, Employees, Employeeroles, Post, Restaurants, Shifts, User, Standardwages
from dmrApp.main.utils import shiftDataReport, payrollReportUtil, dmrDataReport, shiftDataLists,shiftDataLists2,formatExcelHeader
import numpy as np
from datetime import date, datetime
from datetime import timedelta
import datetime
from sqlalchemy import func
import pandas as pd
import os
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
import xlsxwriter

C:\tools\miniconda3\envs\dmrAppConda\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
#dateFrom, dateTo are datetime objects
def shiftDataPayStubReport(dateFrom,dateTo, resId, colNamesShifts):
    shiftData = Shifts.query
    if dateFrom:
        shiftData = shiftData.filter(Shifts.shiftDate>=dateFrom)
    if dateTo:
        shiftData = shiftData.filter(Shifts.shiftDate<=dateTo)
    if resId:
        shiftData = shiftData.filter(Shifts.restaurantId==resId)

    uniqueEmpIdList = shiftData.with_entities(Shifts.empId).distinct().all()
    dfList=[]
    employeeCharMax=10
    for i in uniqueEmpIdList:
        shiftReportColumns = list(shiftDataLists2(shiftData.filter(Shifts.empId==i[0])))
        if len(max(shiftReportColumns[1], key = len))>employeeCharMax:
            employeeCharMax =len(max(shiftReportColumns[1], key = len))
        dfShifts = pd.DataFrame(list(zip(shiftReportColumns[0],shiftReportColumns[1],
            shiftReportColumns[2],shiftReportColumns[3],shiftReportColumns[4],shiftReportColumns[5],
            shiftReportColumns[6],shiftReportColumns[7],shiftReportColumns[8],shiftReportColumns[9],
            shiftReportColumns[10],shiftReportColumns[11])), columns=colNamesShifts)
        dfList.append(dfShifts)
    return (dfList, employeeCharMax)

In [18]:
dateFrom='2021-01-11'
dateTo='2021-01-21'
dateFromForName = datetime.datetime.strptime(dateFrom,'%Y-%m-%d') #converts string to datetime object
dateFromForName = dateFromForName.strftime('%m.%d.%y')
dateToForName = datetime.datetime.strptime(dateTo,'%Y-%m-%d') #converts string to datetime object
dateToForName = dateToForName.strftime('%m.%d.%y')

reportName=f"Test--MultipleRestaurants{dateFromForName}-{dateToForName}.xlsx"

excelObj=pd.ExcelWriter(os.path.join(proxyApp.root_path, 'static/reports/', reportName))
workbook=excelObj.book
formatDecimals = workbook.add_format({'num_format': '#,##0.00'})

sheetName='Payroll Stubs'
resId=7
colNamesShifts =['Date','Name','Restaurant','Role','Scheduled time','Start time',
    'Off time','Hours worked','Tips', 'Wages', 'Tips + Wages','Tips/Hour']
colNamesShiftsStub =['Date','Restaurant','Role','Scheduled time','Start time',
    'Off time','Hours worked','Tips', 'Wages', 'Tips + Wages','Tips/Hour']

In [19]:
payrollStubsDf, employeeCharMax=shiftDataPayStubReport(dateFrom,dateTo, resId, colNamesShifts)
startRow=1
# payrollStubsDf[0].to_excel(excelObj,sheet_name=sheetName, startrow=startRow,header=False, index=False)

In [20]:
startRow=1

dfStubSheet=excelObj.sheets[sheetName]
payrollStubsDfTemp=payrollStubsDf[0][colNamesShiftsStub].copy()
stubName=payrollStubsDf[0].iloc[0,1]
dfStubSheet.write(startRow, 0,stubName)
formatExcelHeader(workbook,dfStubSheet, payrollStubsDfTemp, startRow)


excelObj.close()

KeyError: 'Sheet1'

In [5]:
payrollStubsDfTemp=payrollStubsDf[0][colNamesShiftsStub].copy()

In [6]:
payrollStubsDfTemp

,Date,Restaurant,Role,Scheduled time,Start time,Off time,Hours worked,Tips,Wages,Tips + Wages,Tips/Hour
0,2021-01-18,Hops N Shine,Bartender,None,10:38 AM,09:52 PM,11.233333,405.930000,23.9270,429.857000,36.136202
1,2021-01-19,Hops N Shine,Bartender,None,05:07 PM,10:26 PM,5.316667,107.682741,11.3245,119.007241,20.253807
